In [1]:
from twitter import Twitter

In [2]:
twitter = Twitter()

In [3]:
user_id = 783214
friends_ids = twitter.get_friends_ids(user_id, max_count=100)
followers_ids = twitter.get_followers_ids(user_id, max_count=100)
friends_list = twitter.get_friends_list(user_id, max_count=100)
followers_list = twitter.get_followers_list(user_id, max_count=100)
favorites = twitter.get_favorites(user_id)
lists = twitter.get_lists(user_id)
tweets = twitter.get_tweets(user_id)